In [ ]:
import torch
from transformers import TextClassificationPipeline, AutoModelForSequenceClassification, AutoTokenizer, AutoModel


In [ ]:
model_name = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 3)


In [ ]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)

posts = []
posts.append("Bitcoin is going Up.  Buy, BUy, BUy")
posts.append("Bitcoin #BTC is Down. Bad")
preds = pipe(posts)

print(preds)

In [ ]:
sentence = "Bitcoin #BTC is going Up.  it is great"
sentence1 = "Bitcoin #BTC is Down. Bad"
labels =["Bearish", "Neutral", "Bullish"]

inputs = tokenizer.batch_encode_plus([sentence, sentence1],
                                    return_tensors='pt', max_length=256,
                                    truncation=True, padding='max_length'
                                    )

In [ ]:
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

input_ids.shape, attention_mask.shape

print(attention_mask[1])

In [ ]:
class PyTorch_to_TorchScript(torch.nn.Module):
    def __init__(self):
        super(PyTorch_to_TorchScript, self).__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name,  num_labels = 3)
    def forward(self, data, attention_mask=None):
        return self.model(data, attention_mask)[0]
# after trace it will save the model in cwd
pt_model = PyTorch_to_TorchScript().eval()

# remove_attributes = []
# for key, value in vars(pt_model).items():
#     if value is None:
#         print(f'remove_attributes {key}')
#         remove_attributes.append(key)

# for key in remove_attributes:
#     delattr(pt_model, key)

traced_script_module = torch.jit.trace(pt_model, (input_ids, attention_mask), strict=False)
traced_script_module.save("./model.pt")

In [ ]:
configuration = """
name: "bitcoin-model"
platform: "pytorch_libtorch"
max_batch_size: 32
input [
 {
    name: "input__0"
    data_type: TYPE_INT32
    dims: [ 256 ]
  } ,
{
    name: "input__1"
    data_type: TYPE_INT32
    dims: [ 256 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 3 ]
  }
"""

with open('./config.pbtxt', 'w') as file:
    file.write(configuration)